In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import csv

In [84]:
df1 = pd.read_csv('/Users/hmead/Desktop/BW2/General characteristics of people.csv', sep=';',engine='python', error_bad_lines=False, encoding= 'latin')
df2 = pd.read_csv('/Users/hmead/Desktop/BW2/Datos de la vivienda.csv', sep=';',engine='python', error_bad_lines=False, encoding= 'latin')
df3 = pd.read_csv('/Users/hmead/Desktop/BW2/Perception of security and coexistence.csv', sep=';',engine='python', error_bad_lines=False, encoding= 'latin')

In [94]:
general = df1.copy()
housing = df2.copy()
security = df3.copy()

In [95]:
general_columns = {
            'ï»¿DIRECTORIO':'id', 'SECUENCIA_ENCUESTA':'survey_sequence','SECUENCIA_P':'sequence_p',
            'ORDEN':'order', 'P220' : 'sex', 'P5785': 'age', 'P5501':'relationship_to_head_of_household',
            'P6210':'education_level','P1366': 'relationship_status','P1402':'time_lived_in_city',
            'P1403':'time_lived_in_neighborhood','P1365': 'activity','P1364':'activity_location',
            'P1363': 'activity_time_of_day', 'FEX_C': 'gen_expansion'
              }

housing_columns = {
        'ï»¿DIRECTORIO':'id','SECUENCIA_ENCUESTA':'survey_sequence','SECUENCIA_P':'sequence_p',
        'ORDEN':'order', 'P5747': 'dwelling_type','P5752':'electricity_service', 
        'P5752S1':'social_stratum', 'P5090':'own_rent_usufruct_other','DEPMUNI':'dept_muni',
        'FEX_C': 'housing_expansion'
          }

security_columns = {
          'ï»¿DIRECTORIO':'id','SECUENCIA_ENCUESTA':'survey_sequence','SECUENCIA_P':'sequence_p',
          'P1362':'general_safe_unsafe', 'P1359':'city_safe_unsafe', 'P1594': 'city_security_inc_dec_same',
          'P564':'will_be_victim_next_12_months', 'P562':'how_unsafe', 'P1356S1': 'activity_location_safe_unsafe', 
          'P1356S2':'public_parks_recreational_sports_spaces', 'P1356S3':'market_squares_shopping_streets', 
          'P1356S4': 'public_transportation', 'P1356S5':'ATMs', 'P1356S6':'public_roads', 'P1356S8':'pedestrian_bridges', 
          'P1356S7':'nightclubs_bars_entertainment_venues', 'P1116':'measures_taken', 'P1353S1':'justify_violence_family', 
          'P1353S2':'justify_violence_punishment', 'P1353S3':'justify_violence_self-defense',
          'P1117':'how_certain_will_be_victim_next_12_months', 
          'P1353S4':'justify_violence_infidelity', 'P1353S5':'justify_violence_defend_stranger', 
          'P1353S6':'justify_violence_defend_property', 'P1115': 'action_taken_as_witness', 'FEX_C': 'sec_expansion'
            }

In [96]:
general = general.rename(columns = general_columns)
housing = housing.rename(columns = housing_columns)
security = security.rename(columns = security_columns)

In [97]:
general.head()
# housing.head()
#security.head()

,id,survey_sequence,sequence_p,order,sex,age,relationship_to_head_of_household,education_level,P6210S1,relationship_status,time_lived_in_city,time_lived_in_neighborhood,activity,activity_location,activity_time_of_day,gen_expansion
0,225001,1,1,1,1,48,1,3,304,2,4,4,1,5,1,"372,094531330457"
1,225001,1,2,2,2,55,2,3,302,2,4,4,4,,,"372,094531330457"
2,225001,1,3,3,1,31,3,5,511,6,4,4,1,6,3,"372,094531330457"
3,225002,1,1,1,1,26,1,6,603,6,3,1,1,6,1,"354,84561144316"
4,225002,1,2,2,2,25,7,5,511,6,3,1,3,,,"354,84561144316"


In [98]:
# Housing dataset only has data from:

   # 'sequence_p' ==1        &
   # 'survey_sequence']==1

# Security dataset only has data from:
   # 'age' >14

condition1 = general['age']>14

condition2_g = general['sequence_p'] ==1
condition3_g = general['survey_sequence']==1

condition2_s = security['sequence_p'] ==1
condition3_s = security['survey_sequence']==1

# Apply conditions, in preparation for horizontal merge

general = general[condition1]
general = general[condition2_g]
general = general[condition3_g]

security = security[condition2_s]
security = security[condition3_s]

# Confirm rows

print(general.shape)
print(housing.shape)
print(security.shape)

(64406, 16)
(64406, 10)
(64406, 49)


<ipython-input-98-6e7fac3e2cd4>:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  general = general[condition2_g]
<ipython-input-98-6e7fac3e2cd4>:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  general = general[condition3_g]
<ipython-input-98-6e7fac3e2cd4>:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  security = security[condition3_s]


In [101]:
# Drop Columns with lots of NaNs

general_drop_columns = ['P6210S1', 'gen_expansion', 'survey_sequence', 'sequence_p']

housing_drop_columns = ['housing_expansion', 'survey_sequence', 'sequence_p']

security_drop_columns = ['P1361S1','P1361S2','P1361S3','P1361S4','P1361S5','P1361S6',
                         'P1361S7','P1361S8','P1361S9', 'P563','P1358S1','P1358S10',
                         'P1358S2','P1358S3','P1358S4','P1358S5','P1358S6','P1358S7',
                         'P1361S1', 'P1361S2', 'P1361S3', 'P1361S4', 'survey_sequence',
                         'sequence_p','P1358S8', 'P1358S9', 'P1361S10','P1358S11', 'P1361S10','ORDEN']

In [102]:
general = general.drop(general_drop_columns, axis=1)

housing = housing.drop(housing_drop_columns, axis=1)

security = security.drop(security_drop_columns, axis=1)

In [103]:
print(general.shape)
print(housing.shape)
print(security.shape)

(64406, 12)
(64406, 7)
(64406, 24)


In [104]:
# Merge into one dataframe

df = general.merge(housing, on='id').merge(security, on='id')

df.shape

(64406, 41)

In [105]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        df = df.drop(col,axis=1)

In [106]:
columns = df.dtypes.index.to_list()

In [107]:
df.general_safe_unsafe.value_counts(normalize=True)

1    0.713319
2    0.286681
Name: general_safe_unsafe, dtype: float64

In [108]:
df.action_taken_as_witness.value_counts(normalize=True)

2    0.709872
1    0.151290
4    0.109959
3    0.028879
Name: action_taken_as_witness, dtype: float64

In [109]:
df.nunique()

id                                           64406
sex                                              2
age                                             89
education_level                                  7
relationship_status                              6
time_lived_in_city                               4
time_lived_in_neighborhood                       4
activity                                         8
activity_location                               10
activity_time_of_day                             4
dwelling_type                                    4
electricity_service                              2
social_stratum                                   9
own_rent_usufruct_other                          4
dept_muni                                       28
general_safe_unsafe                              2
how_unsafe                                       4
city_safe_unsafe                                 2
will_be_victim_next_12_months                    2
how_certain_will_be_victim_next

In [78]:
df.activity.value_counts(normalize = True)

1    0.653961
4    0.180822
6    0.079682
2    0.026271
7    0.019191
3    0.017607
5    0.015573
8    0.006894
Name: activity, dtype: float64

In [41]:
df.will_be_victim_next_12_months.value_counts(normalize = True)

2    0.583579
1    0.416421
Name: will_be_victim_next_12_months, dtype: float64